# JEM207 midterm

### Midterm 5.4.2022

In [2]:
#packages that might come in handy
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import time
import os
#from IPython.core.display import HTML, display
idx = pd.IndexSlice

### Part One

In [10]:
link = 'https://ies-midterm.soulution.rocks/'

endpoint = 'login'
my_number = {"cuni": "98698413"}
r = requests.post(f'{link}{endpoint}', json = my_number).json()
print(r)
print("my personal code:", r['data']['personal_code'])
dataset_ids = r['data']['dataset_ids']
dataset_ids

{'data': {'personal_code': '7bb6510e', 'dataset_ids': ['95ff551368', '40bb66faec', '86c214b421', 'f45ec2e9fc', '64a5d99db0', '24f88cb68c', 'e2beb850b2', '8c34bdfc03', '11f651d1ee']}, 'message': 'Hello, Křenková Pavlína! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook.'}
my personal code: 7bb6510e


['95ff551368',
 '40bb66faec',
 '86c214b421',
 'f45ec2e9fc',
 '64a5d99db0',
 '24f88cb68c',
 'e2beb850b2',
 '8c34bdfc03',
 '11f651d1ee']

In [16]:
endpoint = 'data/'
data_list = []
for code in dataset_ids:
    req = requests.get(f'{link}{endpoint}{code}')
    while req.status_code != 200:
        req = requests.get(f'{link}{endpoint}{code}')
    data_list.append(req.json())
data_list[0] #to see the structure

{'data': {'company': 'AHT',
  'data': [['2019-01-02',
    '39.031620025634766',
    '40.90909194946289',
    '38.6363639831543',
    '40.612648010253906',
    '37.26601791381836',
    '87100'],
   ['2019-01-03',
    '40.0197639465332',
    '40.90909194946289',
    '39.525691986083984',
    '39.62450408935547',
    '36.35929870605469',
    '61900'],
   ['2019-01-04',
    '40.0197639465332',
    '43.4782600402832',
    '40.0197639465332',
    '43.083003997802734',
    '39.53281021118164',
    '86200'],
   ['2019-01-07',
    '43.083003997802734',
    '44.36758804321289',
    '42.49011993408203',
    '44.169960021972656',
    '40.53019714355469',
    '76800'],
   ['2019-01-08',
    '44.07114791870117',
    '46.146244049072266',
    '43.37944793701172',
    '45.84980392456055',
    '42.07161331176758',
    '62100'],
   ['2019-01-09',
    '46.44268798828125',
    '47.33201599121094',
    '45.25691604614258',
    '47.03557205200195',
    '43.15966796875',
    '76600'],
   ['2019-01-10',
    '

In [18]:
print(data_list[0]['message'])
company_names = []
for one_comp in data_list:
    company_names.append(one_comp['data']['company'])
company_names #We have 9 companies

Columns are in order Date,Open,High,Low,Close,Adj.Close,Volume. Make sure your data are ordered.


['AHT', 'GSK', 'TSCO', 'AAL', 'LAND', 'PSN', 'IAG', 'IHG', 'MNG']

### Part 2

In [39]:
#creating the dataframe
def Create_PD_series(json1):
    json2 = pd.read_json(json.dumps(json1['data']['data']))
    json2.columns =['Date','Open','High','Low','Close','Adj.Close','Volume'] #from message
    json2 = json2.set_index('Date')
    json2 = json2.stack()
    return json2
series_list = []
for x in data_list:
    series_list.append(Create_PD_series(x))
df = pd.concat(series_list, axis = 1)
df.columns = company_names

AHT           GSK          TSCO           AAL  \
Date                                                                           
2019-01-02 Adj.Close  3.726602e+01  3.432425e+01  8.010709e+01  3.196316e+01   
           Close      4.061265e+01  3.788000e+01  8.223000e+01  3.248000e+01   
           High       4.090909e+01  3.811000e+01  8.401000e+01  3.265000e+01   
           Low        3.863636e+01  3.760000e+01  8.086000e+01  3.105000e+01   
           Open       3.903162e+01  3.777000e+01  8.181000e+01  3.146000e+01   
...                            ...           ...           ...           ...   
2020-11-20 Close      4.000000e+00  3.703000e+01  1.297000e+02  1.253000e+01   
           High       4.690000e+00  3.720000e+01  1.314000e+02  1.291000e+01   
           Low        3.910000e+00  3.688000e+01  1.284200e+02  1.246000e+01   
           Open       4.460000e+00  3.714000e+01  1.300500e+02  1.280000e+01   
           Volume     2.201800e+06  4.273900e+06  1.324500e+06  5.849110e+07   

                               LAND            PSN         IAG            IHG  \
Date                                                                            
2019-01-02 Adj.Close      10.799848            NaN        3.69      51.824001   
           Close          11.700000            NaN        3.69      56.438354   
           High           11.750000            NaN        3.76      56.902000   
           Low            11.190000            NaN        3.60      55.732349   
           Open           11.450000            NaN        3.68      56.817703   
...                             ...            ...         ...            ...   
2020-11-20 Close          15.030000      31.750000        3.41      59.669998   
           High           15.100000      32.147999        3.51      60.130001   
           Low            14.850000      31.575001        3.40      59.549999   
           Open           14.960000      32.070000        3.42      59.930000   
           Volume     195800.000000  347700.000000  6329600.00  100500.000000   

                            MNG  
Date                             
2019-01-02 Adj.Close  36.660000  
           Close      36.660000  
           High       36.810001  
           Low        34.709999  
           Open       35.630001  
...                         ...  
2020-11-20 Close      26.840000  
           High       27.170000  
           Low        26.650000  
           Open       26.940001  
           Volume      0.000000  

[2868 rows x 9 columns]

In [44]:
#What day was the highest number of shares traded?
volume_df = df.loc[idx[:,'Volume',:]]
volume_df = volume_df.set_index('Date')

KeyError: "None of ['Date'] are in the columns"

In [42]:
#What is the highest and lowest Adj Close price the company recorded? 
Adj_Close_max = {}
Adj_Close_min = {}
for company in company_names:
    Adj_Close_max[company] = df.loc[idx[:,'Adj.Close',:]][company].max()
    Adj_Close_min[company] = df.loc[idx[:,'Adj.Close',:]][company].min()
print("The maxima:", Adj_Close_max) #the maxima per company
print("The minima:", Adj_Close_min) #the minima per company

The maxima: {'AHT': 52.77835464477539, 'GSK': 45.580257415771484, 'TSCO': 154.15875244140625, 'AAL': 36.44142532348633, 'LAND': 16.302597045898438, 'PSN': 45.029998779296875, 'IAG': 5.199999809265137, 'IHG': 68.31763458251953, 'MNG': 45.31999969482422}
{'AHT': 1.2899999618530273, 'GSK': 30.737220764160156, 'TSCO': 66.76319885253906, 'AAL': 9.039999961853027, 'LAND': 9.38074779510498, 'PSN': 25.309999465942383, 'IAG': 1.8799999952316284, 'IHG': 27.81444549560547, 'MNG': 14.930000305175781}
